In [1]:
import requests
import re
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import nltk
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor


In [3]:
beer_train = pd.read_csv('beer_11_20.csv')

In [4]:
beer_train.isnull().sum()

Unnamed: 0              0
Unnamed: 0.1            0
Unnamed: 0.1            0
id                      0
name                    0
abv                    17
ibu                     0
originalGravity        16
isOrganic               0
categoryId              0
styleId                 0
style_name              0
style_description       0
beer_description     2859
srmId                  16
duplicate               0
dtype: int64

In [5]:
beer_train2 = beer_train.dropna(subset = ['abv','originalGravity','srmId'])

In [6]:
beer_train.isnull().sum()

Unnamed: 0              0
Unnamed: 0.1            0
Unnamed: 0.1            0
id                      0
name                    0
abv                    17
ibu                     0
originalGravity        16
isOrganic               0
categoryId              0
styleId                 0
style_name              0
style_description       0
beer_description     2859
srmId                  16
duplicate               0
dtype: int64

In [7]:
beer = pd.read_csv('beers_full2.csv')

C:\Users\antho\Anaconda2\envs\data28\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (189,198,200,212,214,215,217,218,220,221,223,224,226,227,236,242,244,245,254,260,262,263,265,266,268,269,271,275,284,290,292,293,295,296,298,299,301,310,312,313,315,316,318,319,321,322,324,325,327,336,338,347,349,350,352,353,355,356,358,359,361,362,364,365,374,380,382,383,392,398,407,413,422,428,437,443,452,458,460,461,463,464,466,467,469,470,472,473,475) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
beer.shape

(59050, 476)

In [9]:
beer.isnull().sum()

Unnamed: 0                       0
Unnamed: 0.1                     0
id                               0
name                             0
abv                          11010
ibu                          38270
originalGravity              54366
ogMin                         7278
isOrganic                        1
categoryId                    2684
category_name                 2684
styleId                       2684
style_name                    2684
style_description             2868
servingTemperatureDisplay    55576
foodPairings                 55945
description                  23632
labels                       37299
glasswareId                  43274
glass_name                   43274
srmId                        53147
srm_hex                      53147
srmMin                        8215
srmMax                        8260
brewery_id                      29
brewery_name                    29
brewery_description          10961
brewery_established          13687
brewery_locality    

In [10]:

beer['beer_description'] = beer['description']

In [11]:
beers = pd.merge(beer, beer_train, how='outer', on = 'id')

In [12]:
beers.head(3)


,Unnamed: 0_x,Unnamed: 0.1_x,id,name_x,abv_x,ibu_x,originalGravity_x,ogMin,isOrganic_x,categoryId_x,...,ibu_y,originalGravity_y,isOrganic_y,categoryId_y,styleId_y,style_name_y,style_description_y,beer_description_y,srmId_y,duplicate
0,0.0,0.0,cBLTUw,"""18"" Imperial IPA 2",11.10,NaN,NaN,1.080,N,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,ZsQEJt,"""633"" American Pale Ale",6.33,25.0,NaN,1.044,N,3.0,...,25.0,1.044773,N,3.0,25.0,American-Style Pale Ale,American pale ales range from deep golden to c...,"Our first beer has been aptly named ""633"" afte...",9.991054,False
2,2.0,2.0,tmEthz,"""Admiral"" Stache",7.00,23.0,1.071,1.072,N,9.0,...,23.0,1.071000,N,9.0,104.0,Baltic-Style Porter,A true smooth cold-fermented and cold lagered ...,Milwaukee Brewing Co’s take on a classic Europ...,37.000000,True


In [ ]:
for index, row in beers[10000:11000].iterrows():
    print beers[beers['id'] == row['id']].index
    if not row['originalGravity_x'] > 0 and row['originalGravity_y'] > 0:
        beers.loc[index, 'originalGravity_x'] = beers.loc[index, 'originalGravity_y']
    if not row['abv_x'] > 0 and row['abv_y'] > 0:
        beers.loc[index, 'abv_x'] = beers.loc[index, 'abv_y']
    if not row['styleId_x'] > 0 and row['styleId_y'] > 0:
        beers.loc[index, 'styleId_x'] = beers.loc[index, 'styleId_y']
    if not row['style_name_x'] > 0 and row['style_name_y'] > 0:
        beers.loc[index, 'style_name_x'] = beers.loc[index, 'style_name_y']
    if not row['beer_description_x'] > 0 and row['beer_description_y'] > 0:
        beers.loc[index, 'beer_description_x'] = beers.loc[index, 'beer_description_y']
    if not row['styleId_x'] > 0 and row['srmId_y'] > 0:
        beers.loc[index, 'srmId_x'] = beers.loc[index, 'srmId_y']

        

Int64Index([10000], dtype='int64')
Int64Index([10001], dtype='int64')
Int64Index([10002], dtype='int64')
Int64Index([10003], dtype='int64')
Int64Index([10004], dtype='int64')
Int64Index([10005], dtype='int64')
Int64Index([10006], dtype='int64')
Int64Index([10007], dtype='int64')
Int64Index([10008], dtype='int64')
Int64Index([10009], dtype='int64')
Int64Index([10010], dtype='int64')
Int64Index([10011], dtype='int64')
Int64Index([10012], dtype='int64')
Int64Index([10013], dtype='int64')
Int64Index([10014], dtype='int64')
Int64Index([10015], dtype='int64')
Int64Index([10016], dtype='int64')
Int64Index([10017], dtype='int64')
Int64Index([10018], dtype='int64')
Int64Index([10019], dtype='int64')
Int64Index([10020], dtype='int64')
Int64Index([10021], dtype='int64')
Int64Index([10022], dtype='int64')
Int64Index([10023], dtype='int64')
Int64Index([10024], dtype='int64')
Int64Index([10025], dtype='int64')
Int64Index([10026], dtype='int64')
Int64Index([10027], dtype='int64')
Int64Index([10028], 

In [ ]:
beers.to_csv('beers_full2.csv', encoding = 'utf-8')
        

In [26]:
beers.isnull().sum()

Unnamed: 0_x                    33
Unnamed: 0.1_x                  33
id                               0
name_x                          33
abv_x                        11010
ibu_x                        38303
originalGravity_x            51724
ogMin                         7311
isOrganic_x                     34
categoryId_x                  2717
category_name                 2717
styleId_x                     2707
style_name_x                  2707
style_description_x           2901
servingTemperatureDisplay    55609
foodPairings                 55978
description                  23665
labels                       37332
glasswareId                  43307
glass_name                   43307
srmId_x                      53171
srm_hex                      53180
srmMin                        8248
srmMax                        8293
brewery_id                      62
brewery_name                    62
brewery_description          10994
brewery_established          13720
brewery_locality    

### 

In [ ]:
#fit prediction model to fill in srmID
feature_cols = ['ibu','abv','styleId']
X_train = beer_train[feature_cols]
y_train = beer_train['srmId']
treereg = DecisionTreeRegressor(max_features = 3)
bagreg = BaggingRegressor(treereg, n_estimators = 490, bootstrap=True, oob_score=True,)
bagreg.fit(X_train,y_train)

In [ ]:
#fill in missing SRM ID 
done = 0
for index, row in beer.iterrows():
    if row['srmMax'] - row['srmMin'] > 5 and row['abv'] > 0:
        beer.loc[index, 'srmId'] = bagreg.predict(np.array(row[feature_cols]).astype(float))[0]
    if not row['srmId'] > 0 and row['abv'] > 0:
        beer.loc[index, 'srmId'] = bagreg.predict(np.array(row[feature_cols]).astype(float))[0]
        done += 1
        if done % 20 == 0:
            print done

In [ ]:
#fit prediction model to fill in OG
feature_cols = ['ibu','abv','styleId','srmId']
X_train = beer_train[feature_cols]
y_train = beer_train['originalGravity']
randreg = RandomForestRegressor(max_features = 1, n_estimators = 220)
randreg.fit(X_train,y_train)

In [ ]:
#fill in missing OG
done = 0
for index, row in beer.iterrows():
    if row['originalGravity'] == row['ogMin'] and row['abv'] > 0 and row['srmId'] >0:
        beer.loc[index, 'originalGravity'] = randreg.predict(np.array(row[feature_cols]).astype(float))[0]
    if not row['originalGravity'] > 0 and row['abv'] > 0:
        beer.loc[index, 'originalGravity'] = randreg.predict(np.array(row[feature_cols]).astype(float))[0]
        done += 1
        if done % 20 == 0:
            print done

In [ ]:
#fit prediction model to fill in ABV
feature_cols = ['ibu', 'srmId','styleId', 'originalGravity']
X_train = beer_train[feature_cols]
y_train = beer_train['abv']
randreg = RandomForestRegressor(max_features = 2, n_estimators = 400)
randreg.fit(X_train,y_train)

In [ ]:
#fill in missing ABV
done = 0
for index, row in beer.iterrows():
        if not row['abv'] > 0 and row['srmId'] > 0 and row['originalGravity'] > 0:
            beer.loc[index, 'abv'] = randreg.predict(np.array(row[feature_cols]).astype(float))[0]
            done += 1
        if done % 20 == 0:
            print done